
# **Assignment_NLP_Proj1_Madhuri.A**

In [ ]:
## PART A

In [1]:
import pandas as pd
import re
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords


from nltk.stem import PorterStemmer
import string
from nltk.tokenize import RegexpTokenizer
import numpy as np
nltk.download('all')

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


import random
import bs4 as bs
import urllib.request
from nltk.stem import WordNetLemmatizer 
import tensorflow as tensorF 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,f1_score,recall_score,precision_score, confusion_matrix


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Pac

In [124]:
# nltk downloaded (run only once)
nltk.download('stopwords',quiet=True) # stopword library
nltk.download('wordnet', quiet=True) # wordnet library
nltk.download('words', quiet=True) # words library
nltk.download('punkt', quiet=True) # tokenize library

True

In [ ]:
##1.Read and Analyse Dataset. [5 Marks]

In [2]:
from google.colab import drive

In [3]:
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [4]:
project_path = ''
dataset_file = project_path + '/content/drive/My Drive/Colab Notebooks/blogs.zip'

In [5]:
from zipfile import ZipFile
with ZipFile(dataset_file,'r') as zip:
  zip.extractall()

In [6]:
!ls

blogtext.csv  drive  sample_data


In [7]:
blogtext_df = pd.read_csv('blogtext.csv',na_values = ' ')
print(blogtext_df.shape)


(681284, 7)


In [8]:
blogtext_df.columns


Index(['id', 'gender', 'age', 'topic', 'sign', 'date', 'text'], dtype='object')

In [9]:
blogtext_df.dtypes


id         int64
gender    object
age        int64
topic     object
sign      object
date      object
text      object
dtype: object

In [10]:
blogtext_df.head(100)

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...
...,...,...,...,...,...,...,...
95,4172416,female,25,indUnk,Capricorn,"08,August,2004","urlLink im new to this, ..."
96,4172416,female,25,indUnk,Capricorn,"08,August,2004",Election time has rolled aro...
97,3668238,female,17,Student,Gemini,"30,June,2004",http://www.uploadimages.net/i...
98,3668238,female,17,Student,Gemini,"26,June,2004",it was fun :) Hey dad ...


In [11]:
group_by_class_df = blogtext_df.groupby(['topic'])
group_by_class_df.head(100)

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...
...,...,...,...,...,...,...,...
157994,3468490,male,14,Environment,Sagittarius,"06,June,2004",urlLink paLawaN beacH urlLink ...
172566,3588087,female,26,Environment,Pisces,"03,July,2004",urlLink This is part of the grounds...
172567,3588087,female,26,Environment,Pisces,"03,July,2004",urlLink There is a boat you can tak...
172568,3588087,female,26,Environment,Pisces,"03,July,2004","urlLink Our first evening there, we..."


In [12]:
len(blogtext_df['id'].unique())

19320

In [13]:
blogtext_df['topic'].unique()

array(['Student', 'InvestmentBanking', 'indUnk', 'Non-Profit', 'Banking',
       'Education', 'Engineering', 'Science', 'Communications-Media',
       'BusinessServices', 'Sports-Recreation', 'Arts', 'Internet',
       'Museums-Libraries', 'Accounting', 'Technology', 'Law',
       'Consulting', 'Automotive', 'Religion', 'Fashion', 'Publishing',
       'Marketing', 'LawEnforcement-Security', 'HumanResources',
       'Telecommunications', 'Military', 'Government', 'Transportation',
       'Architecture', 'Advertising', 'Agriculture', 'Biotech',
       'RealEstate', 'Manufacturing', 'Construction', 'Chemicals',
       'Maritime', 'Tourism', 'Environment'], dtype=object)

In [ ]:
#A.Clearly write outcome of data analysis(Minimum 2 points) [2 Marks]

In [14]:
print("Observations/Analysis:")
print("1) This consolidated CSV file contains  articles on different topics like student,investment banking,agriculture etc.")
print("2) Details like gender,age, sun sign, date of blog written are stored in addition to the blog text.") 
print("3) Blogger is identified by the id and has written multiple blogs on different topics. Eg: Userid - 3588087, has multiple blogs on environment.")
print("4. There are 19320 bloggers. ")

Observations/Analysis:
1) This consolidated CSV file contains  articles on different topics like student,investment banking,agriculture etc.
2) Details like gender,age, sun sign, date of blog written are stored in addition to the blog text.
3) Blogger is identified by the id and has written multiple blogs on different topics. Eg: Userid - 3588087, has multiple blogs on environment.
4. There are 19320 bloggers. 


In [15]:
#Reducing the same size to .02 fraction for faster execution on the classifier models
# generating one row 
blogs_sample_df = blogtext_df.sample(frac =.02)

In [16]:
blogs_sample_df.shape

(13626, 7)

In [17]:
blogs_sample_df.dtypes

id         int64
gender    object
age        int64
topic     object
sign      object
date      object
text      object
dtype: object

In [ ]:
#B.Clean the Structured Data [3 Marks]

In [ ]:
#i.Missing value analysis and imputation. [1 Marks]

In [18]:
percent_feature_missing = blogs_sample_df.isnull().sum() * 100 / len(blogs_sample_df)
missing_data_value_df = pd.DataFrame({'column_name': blogs_sample_df.columns,
                                 'percent_missing': percent_feature_missing})
missing_data_value_df

#print("No missing data")

,column_name,percent_missing
id,id,0.0
gender,gender,0.0
age,age,0.0
topic,topic,0.0
sign,sign,0.0
date,date,0.0
text,text,0.0


In [19]:
print("No missing data")

No missing data


In [132]:
#ii.Eliminate Non-English textual data. [2 Marks]
#Hint: Refer ‘langdetect’ library to detect language of the input text)

In [20]:
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
from langdetect import detect

def detect_english(text):
  try:
      return detect(text) == 'en'
  except:
      return False

In [22]:
blogs_sample_df = blogs_sample_df[blogs_sample_df['text'].apply(detect_english)]

In [23]:
blogs_sample_df.shape

(13007, 7)

In [ ]:
##2. Preprocess unstructured data to make it consumable for model training. [5 Marks]


In [ ]:
#A.Eliminate All special Characters and Numbers [2 Marks]


In [24]:
def remove_special_chars_and_digits(text):
    return re.sub("(\\d|\\W)+"," ", text)

In [25]:
def normalize_text(text: str) -> str:
    text = remove_special_chars_and_digits(text)
    return text

In [26]:
# Creating a new feature with normalized text

blogs_sample_df['normalized_text'] = blogs_sample_df['text'].apply(normalize_text)
blogs_sample_df.loc[:, ['text', 'normalized_text']].head(50)

,text,normalized_text
538461,"I wasn't going to do this, but I ...",I wasn t going to do this but I have no choic...
340758,Im looking at this fine dir...,Im looking at this fine director and thinking...
445747,"Well, this was rather surprising. I we...",Well this was rather surprising I went to pos...
652503,It was the last one. We picked it up n ...,It was the last one We picked it up n started...
568568,now for a random thoughtful excerpt...,now for a random thoughtful excerpt from a ch...
371507,I totally forgot that I had won (again)...,I totally forgot that I had won again two tic...
157822,&nbsp;&nbsp;&nbsp;&nbsp;&nbsp...,nbsp nbsp nbsp nbsp nbsp I ask that you all b...
53976,Today Marie called and said that ...,Today Marie called and said that her relation...
303516,"Just for all of you wondering, the old ...",Just for all of you wondering the old Amy Luh...
258329,yay im really happy rite now. For chris...,yay im really happy rite now For christmas se...


In [ ]:
#B.Lowercase all textual data [1 Marks]


In [27]:

def lower_case(text):
    return text.lower()    

In [28]:
def normalize_lower_case(text: str) -> str:
    text = lower_case(text)
 
    return text

In [29]:

blogs_sample_df['normalized_text'] = blogs_sample_df['normalized_text'].apply(normalize_lower_case)
blogs_sample_df.loc[:, ['text', 'normalized_text']].head(50)

,text,normalized_text
538461,"I wasn't going to do this, but I ...",i wasn t going to do this but i have no choic...
340758,Im looking at this fine dir...,im looking at this fine director and thinking...
445747,"Well, this was rather surprising. I we...",well this was rather surprising i went to pos...
652503,It was the last one. We picked it up n ...,it was the last one we picked it up n started...
568568,now for a random thoughtful excerpt...,now for a random thoughtful excerpt from a ch...
371507,I totally forgot that I had won (again)...,i totally forgot that i had won again two tic...
157822,&nbsp;&nbsp;&nbsp;&nbsp;&nbsp...,nbsp nbsp nbsp nbsp nbsp i ask that you all b...
53976,Today Marie called and said that ...,today marie called and said that her relation...
303516,"Just for all of you wondering, the old ...",just for all of you wondering the old amy luh...
258329,yay im really happy rite now. For chris...,yay im really happy rite now for christmas se...


In [ ]:
#C.Remove all Stopwords [1 Marks]

In [30]:
def stopword_lemma(text):
    token = nltk.word_tokenize(text)
    text_stop = [x for x in token if x not in set(stopwords.words('english'))]
    lemmatizer = WordNetLemmatizer()
    text_lemma = [lemmatizer.lemmatize(word) for word in text_stop]
    text_lemma = ' '.join(text_lemma)
    return text_lemma

In [31]:
def normalize_remove_stop_words(text: str) -> str:
    text = stopword_lemma(text)

    return text

In [32]:
blogs_sample_df['normalized_text'] = blogs_sample_df['normalized_text'].apply(normalize_remove_stop_words)
blogs_sample_df.loc[:, ['text', 'normalized_text']].head()

,text,normalized_text
538461,"I wasn't going to do this, but I ...",going choice comment fact producer contracted ...
340758,Im looking at this fine dir...,im looking fine director thinking may end list...
445747,"Well, this was rather surprising. I we...",well rather surprising went post comment urlli...
652503,It was the last one. We picked it up n ...,last one picked n started fighting moment enth...
568568,now for a random thoughtful excerpt...,random thoughtful excerpt chat finalsaturnian ...


In [ ]:
#D.Remove all extra white spaces [1 Marks]

In [33]:

blogs_sample_df.normalized_text = blogs_sample_df.normalized_text.replace('\s+', ' ', regex=True)


In [34]:
def remove_whitespace(x):
    """
    Helper function to remove any blank space from a string
    x: a string
    """
    try:
        # Remove spaces inside of the string
        x = " ".join(x.split())
        x.replace(" ","")
        x.strip()
        x.lstrip()
        x.rstrip()

    except:
        pass
    return x


In [35]:

blogs_sample_df.normalized_text = blogs_sample_df.normalized_text.apply(remove_whitespace)

In [36]:
blogs_sample_df.loc[:, ['text', 'normalized_text']].head(10)

,text,normalized_text
538461,"I wasn't going to do this, but I ...",going choice comment fact producer contracted ...
340758,Im looking at this fine dir...,im looking fine director thinking may end list...
445747,"Well, this was rather surprising. I we...",well rather surprising went post comment urlli...
652503,It was the last one. We picked it up n ...,last one picked n started fighting moment enth...
568568,now for a random thoughtful excerpt...,random thoughtful excerpt chat finalsaturnian ...
371507,I totally forgot that I had won (again)...,totally forgot two ticket premiere honey starr...
157822,&nbsp;&nbsp;&nbsp;&nbsp;&nbsp...,nbsp nbsp nbsp nbsp nbsp ask prayer family nbs...
53976,Today Marie called and said that ...,today marie called said relationship screwed r...
303516,"Just for all of you wondering, the old ...",wondering old amy luhnette back alive well cas...
258329,yay im really happy rite now. For chris...,yay im really happy rite christmas seema gave ...


In [ ]:
##3. Build a base Classification model [8 Marks]


In [ ]:
#A.Create dependent and independent variables [2 Marks]Hint: Treat ‘topic’ as a Target variable.


In [37]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
blogs_sample_df['topic'] = le.fit_transform(blogs_sample_df['topic'])
blogs_sample_df.head()

,id,gender,age,topic,sign,date,text,normalized_text
538461,1507424,male,35,4,Cancer,"24,May,2003","I wasn't going to do this, but I ...",going choice comment fact producer contracted ...
340758,4039098,female,24,39,Sagittarius,"12,August,2004",Im looking at this fine dir...,im looking fine director thinking may end list...
445747,3320360,male,27,4,Capricorn,"12,May,2004","Well, this was rather surprising. I we...",well rather surprising went post comment urlli...
652503,3150267,female,16,34,Libra,"27,May,2004",It was the last one. We picked it up n ...,last one picked n started fighting moment enth...
568568,1929559,female,16,39,Gemini,"15,September,2003",now for a random thoughtful excerpt...,random thoughtful excerpt chat finalsaturnian ...


In [38]:
blogs_sample_df.dtypes

id                  int64
gender             object
age                 int64
topic               int64
sign               object
date               object
text               object
normalized_text    object
dtype: object

In [39]:
# Remove two columns name is age,gender and date
blogs_sample_df.drop(['age','gender', 'sign','date'], axis=1)

,id,topic,text,normalized_text
538461,1507424,4,"I wasn't going to do this, but I ...",going choice comment fact producer contracted ...
340758,4039098,39,Im looking at this fine dir...,im looking fine director thinking may end list...
445747,3320360,4,"Well, this was rather surprising. I we...",well rather surprising went post comment urlli...
652503,3150267,34,It was the last one. We picked it up n ...,last one picked n started fighting moment enth...
568568,1929559,39,now for a random thoughtful excerpt...,random thoughtful excerpt chat finalsaturnian ...
...,...,...,...,...
195958,823780,13,"Another bored, bored day. Surfing, n200...",another bored bored day surfing n pre demo dem...
410266,2950525,6,"February 17th, 2004 Leo (...",february th leo july aug take charge today suc...
464929,1284157,34,Women fascinate me Just so...,woman fascinate know squirrel
168485,3871907,21,Colleges in Indiana -- and elsewhere --...,college indiana elsewhere considering whether ...


In [ ]:
#C.Vectorize data using any one vectorizer. [2 Marks]


In [40]:
vectorizer_tfidf = TfidfVectorizer(stop_words = set(nltk.corpus.stopwords.words('english')))
tfidf_method = vectorizer_tfidf.fit_transform(blogs_sample_df.normalized_text)
tfidf_method.shape

(13007, 63001)

In [41]:
# Create Data Frame of tdidf scores
tfidf_df_method = pd.DataFrame(tfidf_method.toarray(),
             columns = vectorizer_tfidf.get_feature_names(),
             index = blogs_sample_df.index)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [42]:
# Calculate tfidf for all columns and list top 10
tfidf_df_method.mean().sort_values(ascending = False).head(10)  

urllink    0.033293
nbsp       0.026003
like       0.023016
one        0.020369
get        0.018835
know       0.018612
time       0.018479
day        0.017161
go         0.016329
really     0.016250
dtype: float64

In [ ]:
#B.Split data into train and test. [1 Marks]


In [43]:
X_train, X_test, y_train, y_test = train_test_split(blogs_sample_df['normalized_text'].values, blogs_sample_df['topic'].values, test_size=0.15, random_state=42)
print('Training utterances: {}'.format(X_train.shape[0]))
print('Validation utterances: {}'.format(X_test.shape[0]))

Training utterances: 11055
Validation utterances: 1952


In [44]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X_train)

TfidfVectorizer()

In [45]:
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)
X_train, X_test

(<11055x57206 sparse matrix of type '<class 'numpy.float64'>'
 	with 838454 stored elements in Compressed Sparse Row format>,
 <1952x57206 sparse matrix of type '<class 'numpy.float64'>'
 	with 144941 stored elements in Compressed Sparse Row format>)

In [47]:
#selecting  the k-best features
from sklearn.feature_selection import SelectKBest, chi2

ch2 = SelectKBest(chi2, k=5000)
X_train = ch2.fit_transform(X_train, y_train)
X_test = ch2.transform(X_test)

X_train, X_test

(<11055x5000 sparse matrix of type '<class 'numpy.float64'>'
 	with 37182 stored elements in Compressed Sparse Row format>,
 <1952x5000 sparse matrix of type '<class 'numpy.float64'>'
 	with 6181 stored elements in Compressed Sparse Row format>)

In [51]:
#D.Build a base model for Supervised Learning - Classification. [2 Marks]


In [48]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
mnb_model1 = MultinomialNB()
mnb_model1.fit(X_train, y_train)
pred = mnb_model1.predict(X_test)
print("MultinomialNB Accuracy =",accuracy_score(y_test, pred))

MultinomialNB Accuracy = 0.35348360655737704


In [ ]:
#E.Clearly print Performance Metrics. [1 Marks]Hint: Accuracy, Precision, Recall, ROC-AUC

In [49]:
from sklearn.metrics import classification_report

print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.00      0.00      0.00        18
           2       0.00      0.00      0.00         6
           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00        87
           5       0.00      0.00      0.00         6
           6       0.00      0.00      0.00        10
           7       0.00      0.00      0.00        11
           8       0.00      0.00      0.00        10
           9       0.00      0.00      0.00        12
          10       0.00      0.00      0.00        75
          11       0.00      0.00      0.00         4
          12       0.00      0.00      0.00        15
          13       0.00      0.00      0.00        89
          14       0.00      0.00      0.00        36
          15       0.00      0.00      0.00         1
          16       0.00      0.00      0.00        16
          17       0.00    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [90]:
print("Observations:")
print("Since the same size is small and distributed among 39 different topics, precision and accuracy is zero for small sample sized topics. ")
print("The model is able to predict with better precision and recall the topics with more representative data in sample.")

Observations:
Since the same size is small and distributed among 39 different topics, precision and accuracy is zero for small sample sized topics. 
The model is able to predict with better precision and recall the topics with more representative data in sample.


In [ ]:
##4.Improve Performance of model. [14 Marks]


In [ ]:
#A.Experiment with other vectorisers. [4 Marks]


In [50]:
# Importing the glove model and creating the embeddings

import os
glove_path = '/content/drive/My Drive/Colab Notebooks/glove.6B.300d.txt'
embeddings_index = {}
f = open(glove_path, encoding='utf8')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
    #print(embeddings_index)
f.close()

In [51]:
# Taking average of all word embeddings in a sentence to generate the sentence representation.
data_list = list()
for comp in blogs_sample_df['normalized_text']:
    sentence = np.zeros(300)
    count = 0
    for w in normalize_text(comp):
        try:
            sentence += embeddings_index[w]
            count += 1
        except KeyError:
            continue
    data_list.append(sentence / count)

len(data_list[0])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


300

In [54]:
blogs_sample_df[:] = np.nan_to_num(blogs_sample_df)

In [55]:
## Train_Test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(np.array(data_list), blogs_sample_df.topic.values, test_size=0.15, random_state=42)
print(X_train.shape, y_train.shape)
X_train = np.nan_to_num(X_train) 


(11055, 300) (11055,)


In [ ]:
#B.Build classifier Models using other algorithms than base model. [4 Marks]


In [56]:
### Training and Testing the GaussianNB classifier 
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
gnb_model1 = GaussianNB()
gnb_model1.fit(X_train, y_train)
pred_gnb = gnb_model1.predict(X_test)
print("GaussianNB accuracy =",accuracy_score(y_test, pred_gnb))

GaussianNB accuracy = 0.017930327868852458


In [57]:
### Training and Testing the BernoulliNB classifier 
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score
bnb_model1 = BernoulliNB()
bnb_model1.fit(X_train, y_train)
pred_bnb = bnb_model1.predict(X_test)
print("BernoulliNB accuracy =",accuracy_score(y_test, pred_bnb))

BernoulliNB accuracy = 0.32223360655737704


In [58]:
### Training and Testing the LogisticRegression classifier 
#Changing the solver method to tune the model
from sklearn.linear_model import LogisticRegression
logreg2 = LogisticRegression(solver='newton-cg', random_state=1,max_iter=1000)

logreg2.fit(X_train, y_train)
pred_logreg2 = logreg2.predict(X_test)
model_score = logreg2.score(X_test, y_test)
print("LogisticRegression accuracy =",model_score)


LogisticRegression accuracy = 0.34989754098360654


In [59]:

### Training and Testing the classifier
## RandomForest model
from sklearn.ensemble import RandomForestClassifier
model_gloveembed_rf=RandomForestClassifier()
model_gloveembed_rf.fit(X_train, y_train)
pred = model_gloveembed_rf.predict(X_test)
print('RF_Accuracy_Score =', accuracy_score(y_test, pred))

RF_Accuracy_Score = 0.3452868852459016


In [60]:

# Building a Support Vector Machine on train data with kernel ="rbf"
svc_model = SVC(kernel='rbf')
svc_model.fit(X_train, y_train)
prediction_svc = svc_model.predict(X_test)

print('SVC accuracy =',svc_model.score(X_test, y_test))

SVC accuracy = 0.35297131147540983


In [ ]:
#C.Tune Parameters/Hyperparameters of the model/s. [4 Marks]


In [61]:
#tuning with max_depth,max_features of RandomForestClassifier
## RandomForest model
from sklearn.ensemble import RandomForestClassifier
model_gloveembed_rf_t=RandomForestClassifier(max_depth=4,max_features=4)
model_gloveembed_rf_t.fit(X_train, y_train)
pred_rf_t = model_gloveembed_rf.predict(X_test)
print('RF accuracy Score =', accuracy_score(y_test, pred_rf_t))

RF accuracy Score = 0.3452868852459016


In [62]:
#Changing the solver method to tune the model
from sklearn.linear_model import LogisticRegression
logreg2_t = LogisticRegression(solver='saga', random_state=1,max_iter=1000)

logreg2_t.fit(X_train, y_train)
pred_logreg2 = logreg2_t.predict(X_test)
model_score = logreg2_t.score(X_test, y_test)
print('LogisticRegression accuracy =',model_score)


LogisticRegression accuracy = 0.34989754098360654


In [63]:
from sklearn.svm import SVC

# Building a Support Vector Machine on train data with kernel ="linear"
svc_model_t = SVC(kernel='linear')
svc_model_t.fit(X_train, y_train)
prediction_svc_t = svc_model_t.predict(X_test)

print('SVC accuracy =',svc_model.score(X_test, y_test))

SVC accuracy = 0.35297131147540983


In [ ]:
#D.Clearly print Performance Metrics. [2 Marks]Hint: Accuracy, Precision, Recall, ROC-AUC©Great Learning. 

In [64]:
print("Classification report for Logistic Regression Model : ")
print(classification_report(y_test, pred_logreg2))

Classification report for Logistic Regression Model : 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.00      0.00      0.00        18
           2       0.00      0.00      0.00         6
           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00        87
           5       0.00      0.00      0.00         6
           6       0.00      0.00      0.00        10
           7       0.00      0.00      0.00        11
           8       0.00      0.00      0.00        10
           9       0.00      0.00      0.00        12
          10       0.00      0.00      0.00        75
          11       0.00      0.00      0.00         4
          12       0.00      0.00      0.00        15
          13       0.00      0.00      0.00        89
          14       0.00      0.00      0.00        36
          15       0.00      0.00      0.00         1
          16       0.00   

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [65]:
print("Low precision and recall for most of the topics is due to small representative data in the sample.")

Low precision and recall for most of the topics is due to small representative data in the sample.


In [66]:
print("Classification report for Randomforest classifier : ")
print(classification_report(y_test, pred_rf_t))

Classification report for Randomforest classifier : 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.00      0.00      0.00        18
           2       0.00      0.00      0.00         6
           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00        87
           5       0.00      0.00      0.00         6
           6       0.00      0.00      0.00        10
           7       0.00      0.00      0.00        11
           8       0.00      0.00      0.00        10
           9       0.00      0.00      0.00        12
          10       1.00      0.01      0.03        75
          11       0.00      0.00      0.00         4
          12       0.00      0.00      0.00        15
          13       0.00      0.00      0.00        89
          14       0.00      0.00      0.00        36
          15       0.00      0.00      0.00         1
          16       0.00     

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [67]:
print("Classification report for SVC classifier : ")
print(classification_report(y_test, prediction_svc))

Classification report for SVC classifier : 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.00      0.00      0.00        18
           2       0.00      0.00      0.00         6
           3       0.00      0.00      0.00         3
           4       0.00      0.00      0.00        87
           5       0.00      0.00      0.00         6
           6       0.00      0.00      0.00        10
           7       0.00      0.00      0.00        11
           8       0.00      0.00      0.00        10
           9       0.00      0.00      0.00        12
          10       0.00      0.00      0.00        75
          11       0.00      0.00      0.00         4
          12       0.00      0.00      0.00        15
          13       0.00      0.00      0.00        89
          14       0.00      0.00      0.00        36
          15       0.00      0.00      0.00         1
          16       0.00      0.00    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
##5.Share insights on relative performance comparison [8 Marks]


In [ ]:
 #A.Which vectorizer performed better? Probable reason?. [2 Marks]


In [68]:
print("Here TfidfVectorizer based model has performed well over Glove embedding based model.")
print("Best results are obtained using the more generic TF-IDF method by a factor of roughly 1-2%. ")
print("Reason being TF-IDF method gains more information from longer documents compared to the embedding method.")


Here TfidfVectorizer based model has performed well over Glove embedding based model.
Best results are obtained using the more generic TF-IDF method by a factor of roughly 1-2%. 
Reason being TF-IDF method gains more information from longer documents compared to the embedding method.


In [ ]:
 #B.Which model outperformed? Probable reason? [2 Marks]
 

In [69]:
print("MultinomialNB based model has performed well based on the nature of the data.")
print("SVC with rbf kernel has almost same accuracy as MultinomialNB.")
print("The algorithm is based on the Bayes theorem and predicts the tag of a text.")
print("It calculates the probability of each tag for a given sample and then gives the tag with the highest probability as output.")

MultinomialNB based model has performed well based on the nature of the data.
SVC with rbf kernel has almost same accuracy as MultinomialNB.
The algorithm is based on the Bayes theorem and predicts the tag of a text.
It calculates the probability of each tag for a given sample and then gives the tag with the highest probability as output.


In [70]:
#C.Which  parameter/hyperparameter  significantly  helped to improve performance?Probable reason?. [2 Marks]
 

In [71]:
print("MultinomialNB model  as the parameter for the model, is best suited for NLP classification problem among all the Naive Bayes classifiers.")
print(" SVC classifier can also be used for comparoitive performance.")

MultinomialNB model  as the parameter for the model, is best suited for NLP classification problem among all the Naive Bayes classifiers.
 SVC classifier can also be used for comparoitive performance.


In [ ]:
#D.According  to  you,  which  performance  metric  should  be given most importance, why?. [2 Marks]     Part A - 40 Marks



In [72]:
print("Precision is the important metric as we want to categorise blogs correctly. ")
print("To handle furture request for a particular topic, the document shoud be correctly classified.")

Precision is the important metric as we want to categorise blogs correctly. 
To handle furture request for a particular topic, the document shoud be correctly classified.


In [ ]:
##PART B

In [ ]:
#PROJECT OBJECTIVE: Design a python based interactive semi - rule based chatbot which can do the following:



In [73]:
import json
  
# Opening JSON file
f = open('/content/drive/My Drive/Colab Notebooks/GLBotCorpus.json')
  
# returns JSON object as 
# a dictionary
corpus_data = json.load(f)
  
# Iterating through the json
# list
for i in corpus_data['intents']:
    print(i)
  
# Closing file
f.close()

{'tag': 'Intro', 'patterns': ['hi', 'how are you', 'is anyone there', 'hello', 'whats up', 'hey', 'yo', 'listen', 'please help me', 'i am learner from', 'i belong to', 'aiml batch', 'aifl batch', 'i am from', 'my pm is', 'blended', 'online', 'i am from', 'hey ya', 'talking to you for first time'], 'responses': ['Hello from GreatLearning!. how can i help you ?'], 'context_set': ''}
{'tag': 'Exit', 'patterns': ['thank you', 'thanks', 'cya', 'see you', 'later', 'see you later', 'goodbye', 'byee', 'i am leaving', 'have a Good day', 'you helped me', 'thanks a lot', 'thanks a ton', 'you are the best', 'great help', 'too good', 'you are a good learning buddy'], 'responses': ['I hope I was able to assist you well, Good Bye'], 'context_set': ''}
{'tag': 'Olympus', 'patterns': ['olympus', 'explain me how olympus works', 'I am not able to understand olympus', 'olympus window not working', 'no access to olympus', 'unable to see link in olympus', 'no link visible on olympus', 'whom to contact for o

In [ ]:
#1. Start chat session with greetings and ask what the user is looking for. [5 Marks]

In [ ]:
#2. Accept dynamic text based questions from the user. Reply back with relevant answer from the designed corpus. [10 Marks]


In [ ]:
#3. End the chat session only if the user requests to end else ask what the user is looking for. Loop continues till the user asks to end it. [5 Marks]

In [74]:
lm = WordNetLemmatizer() 
# lists
ourClasses = []
newWords = []
doc_X = []
doc_Y = []
#  tokenize intents into words and the patterns and their associated tags lists
for intent in corpus_data["intents"]:
    for pattern in intent["patterns"]:
        ournewTkns = nltk.word_tokenize(pattern)
        newWords.extend(ournewTkns)
        doc_X.append(pattern)
        doc_Y.append(intent["tag"])

    # add tags to their respective classes
    if intent["tag"] not in ourClasses:
        ourClasses.append(intent["tag"])

# set words to lowercase if not in punctuation
newWords = [lm.lemmatize(word.lower()) for word in newWords if word not in string.punctuation] 
# sorting words
newWords = sorted(set(newWords))
ourClasses = sorted(set(ourClasses))

In [75]:
trainingData = [] # training list array
outEmpty = [0] * len(ourClasses)
# bow model
for idx, doc in enumerate(doc_X):
    bagOfwords = []
    text = lm.lemmatize(doc.lower())
    for word in newWords:
        bagOfwords.append(1) if word in text else bagOfwords.append(0)

    outputRow = list(outEmpty)
    outputRow[ourClasses.index(doc_Y[idx])] = 1
    trainingData.append([bagOfwords, outputRow])

random.shuffle(trainingData)
trainingData = np.array(trainingData, dtype=object)

x = np.array(list(trainingData[:, 0]))
y = np.array(list(trainingData[:, 1]))

In [76]:
# defining a Neural Network classifier
iShape = (len(x[0]),)
oShape = len(y[0])
# parameter definition
bot_model = Sequential()

bot_model.add(Dense(128, input_shape=iShape, activation="relu"))
bot_model.add(Dropout(0.5))

bot_model.add(Dense(64, activation="relu"))
bot_model.add(Dropout(0.3))
bot_model.add(Dense(oShape, activation = "softmax"))

md = tensorF.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)


In [77]:
#compiling the model
bot_model.compile(loss='categorical_crossentropy',
              optimizer=md,
              metrics=["accuracy"])

# model summary
print(bot_model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               20480     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 8)                 520       
                                                                 
Total params: 29,256
Trainable params: 29,256
Non-trainable params: 0
_________________________________________________________________
None


In [78]:
#training the model
bot_model.fit(x, y, epochs=200, verbose=1)

Epoch 1/200
5/5 [==============================] - 1s 8ms/step - loss: 2.0490 - accuracy: 0.1705
Epoch 2/200
5/5 [==============================] - 0s 4ms/step - loss: 1.7979 - accuracy: 0.2558
Epoch 3/200
5/5 [==============================] - 0s 5ms/step - loss: 1.5711 - accuracy: 0.4884
Epoch 4/200
5/5 [==============================] - 0s 4ms/step - loss: 1.3325 - accuracy: 0.5891
Epoch 5/200
5/5 [==============================] - 0s 4ms/step - loss: 1.0947 - accuracy: 0.6124
Epoch 6/200
5/5 [==============================] - 0s 6ms/step - loss: 0.9402 - accuracy: 0.6512
Epoch 7/200
5/5 [==============================] - 0s 4ms/step - loss: 0.6637 - accuracy: 0.7984
Epoch 8/200
5/5 [==============================] - 0s 4ms/step - loss: 0.5928 - accuracy: 0.8295
Epoch 9/200
5/5 [==============================] - 0s 5ms/step - loss: 0.4256 - accuracy: 0.8992
Epoch 10/200
5/5 [==============================] - 0s 4ms/step - loss: 0.3850 - accuracy: 0.8605
Epoch 11/200
5/5 [===========

In [79]:
#tokenizing entered text
def tokenizeNewText(text):
  new_tokns = nltk.word_tokenize(text)
  new_tokns = [lm.lemmatize(word) for word in new_tokns]
  return new_tokns

#creating counts for words in msg
def wordBag(msgtext, vocab):
  new_tokens = tokenizeNewText(msgtext)
  bagOwords = [0] * len(vocab)
  for t in new_tokens:
    for idx, word in enumerate(vocab):
      if word == t:
        bagOwords[idx] = 1
  return np.array(bagOwords)

#making predictions using the sequential model defined above.
def bag_of_words_model(msgtext, vocab, labels):
  newThreshold = 0.3
  bagOwords = wordBag(msgtext, vocab)
  result = bot_model.predict(np.array([bagOwords]))[0]
  yp = [[idx, res] for idx, res in enumerate(result) if res > newThreshold]

  yp.sort(key=lambda x: x[1], reverse=True)
  newList = []
  for r in yp:
    newList.append(labels[r[0]])
  return newList

def bot_Response(firstlist, fJson):
  tag = firstlist[0]
  listOfIntents = fJson["intents"]
  for i in listOfIntents:
    if i["tag"] == tag:
      result = random.choice(i["responses"])
      break
  return result

In [80]:
continue_flag = True
print("*********************************************************************************")
print("*********************************************************************************")
print(" Welcome!! This is GreatLearning Chatbot. Please type 'Exit' to end this session.")

while (continue_flag == True):
  newMessage = input("")
  if newMessage != 'Exit':
      intents = bag_of_words_model(newMessage, newWords, ourClasses)
      response = bot_Response(intents, corpus_data)
      print(response)
  else:
    print("Ending the session.")
    continue_flag = False

*********************************************************************************
*********************************************************************************
 Welcome!! This is GreatLearning Chatbot. Please type 'Exit' to end this session.
hi
1/1 [==============================] - 0s 103ms/step
Hello from GreatLearning!. how can i help you ?
machine learning
1/1 [==============================] - 0s 19ms/step
Refer to Machine Learning links on wiki 
Exit
Ending the session.
